In [1]:
from dateutil import parser
from datetime import datetime, UTC
from zoneinfo import ZoneInfo, available_timezones

In [2]:
dt = datetime.now(tz=UTC)
dt

datetime.datetime(2025, 7, 23, 20, 24, 25, 951797, tzinfo=datetime.timezone.utc)

In [7]:
for zone in available_timezones():
    print(zone)

GMT-0
Pacific/Funafuti
Atlantic/St_Helena
Pacific/Pago_Pago
America/Cayman
Asia/Kathmandu
Europe/Tiraspol
America/Monterrey
America/Mazatlan
Asia/Khandyga
America/Cordoba
America/Tegucigalpa
Indian/Chagos
Pacific/Samoa
Asia/Ulan_Bator
Africa/Nairobi
Canada/Yukon
Pacific/Efate
America/Sao_Paulo
Singapore
America/Argentina/San_Juan
Asia/Tbilisi
Asia/Shanghai
Africa/Nouakchott
Pacific/Easter
Pacific/Midway
Pacific/Marquesas
Asia/Tehran
Europe/Stockholm
America/St_Lucia
Europe/London
Africa/Abidjan
Atlantic/Faroe
America/St_Thomas
Africa/Timbuktu
America/Whitehorse
America/Porto_Acre
Asia/Calcutta
America/Puerto_Rico
America/Punta_Arenas
US/East-Indiana
Australia/West
Asia/Bangkok
Asia/Aqtau
Pacific/Wallis
Etc/GMT+6
Asia/Pyongyang
Etc/GMT-1
MST
America/Kentucky/Monticello
US/Samoa
America/Panama
Etc/GMT+12
Etc/GMT+8
Europe/Istanbul
America/Guyana
America/Ciudad_Juarez
Canada/Central
Pacific/Kwajalein
America/Argentina/Buenos_Aires
Antarctica/Vostok
Arctic/Longyearbyen
America/Juneau
Austra

In [20]:
naive_dt = datetime.now()
naive_dt

datetime.datetime(2025, 7, 21, 0, 57, 29, 171308)

In [21]:
aware_dt = naive_dt.replace(tzinfo=UTC)

In [22]:
aware_dt

datetime.datetime(2025, 7, 21, 0, 57, 29, 171308, tzinfo=datetime.timezone.utc)

In [15]:
aware_dt = naive_dt.replace(tzinfo=ZoneInfo("Europe/Athens"))

In [16]:
aware_dt

datetime.datetime(2025, 7, 21, 0, 40, 43, 71946, tzinfo=zoneinfo.ZoneInfo(key='Europe/Athens'))

In [17]:
utc_dt = aware_dt.astimezone(ZoneInfo("UTC"))

In [18]:
utc_dt

datetime.datetime(2025, 7, 20, 21, 40, 43, 71946, tzinfo=zoneinfo.ZoneInfo(key='UTC'))

In [28]:
from typing import Annotated
from pydantic import BaseModel, BeforeValidator, AfterValidator

def parse_datetime(value: str) -> datetime:
    if isinstance(value, str):
        try:
            return parser.parse(value, dayfirst=True)
        except Exception as ex:
            raise ValueError(ex)
    return value

def to_athens_zoneinfo(dt: datetime) -> ZoneInfo:
    # If datetime is naive, assume that the timezone if UTC
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=UTC)

    athens_dt = dt.astimezone(ZoneInfo("Europe/Athens"))
    return athens_dt


AthensDateTime = Annotated[datetime,
                           BeforeValidator(parse_datetime),
                           AfterValidator(to_athens_zoneinfo)
                            ]

# --- Pydantic Model ---
class Event(BaseModel):
    name: str
    timestamp: AthensDateTime

# 1. Naive datetime string (assumed UTC, converted to Athens)
e1 = Event(name="Meeting", timestamp="2025-07-20 12:00")
print(e1.timestamp)  # 2025-07-20 15:00:00+03:00

# 2. Aware datetime string in UTC
e2 = Event(name="Call", timestamp="2025-07-20T10:00:00+00:00")
print(e2.timestamp)  # 2025-07-20 13:00:00+03:00

# 3. Aware datetime string in Athens time
e3 = Event(name="Lunch", timestamp="2025-07-20T13:00:00+03:00")
print(e3.timestamp)  # 2025-07-20 13:00:00+03:00 (unchanged)

2025-07-20 15:00:00+03:00
2025-07-20 13:00:00+03:00
2025-07-20 13:00:00+03:00


In [31]:
# Format: "July 20, 2025 2:15 PM"
e1 = Event(name="Friendly reminder", timestamp="July 20, 2025 2:15 PM")
print(e1.timestamp)

# Format: "20/07/2025 14:15"
e2 = Event(name="European format", timestamp="20/07/2025 14:15")
print(e2.timestamp)

# Format: "2025.07.20 AD at 14:15:00"
e3 = Event(name="Fancy format", timestamp="2025.07.20 AD at 14:15:00")
print(e3.timestamp)

# Format: "Sunday, July 20, 2025"
e4 = Event(name="All-day", timestamp="Sunday, July 20, 2025")
print(e4.timestamp)

# Format: "07-20-2025 2:15PM"
e5 = Event(name="US format", timestamp="07-20-2025 2:15PM")
print(e5.timestamp)

# Format: "07-20-2025 2:15PM"
e6 = Event(name="US format", timestamp="07-08-2025 2:15PM")
print(e6.timestamp)

# Format: "07-20-2025 2:15PM"
e7 = Event(name="US format", timestamp="2025-07-08 2:15PM")
print(e6.timestamp)

2025-07-20 17:15:00+03:00
2025-07-20 17:15:00+03:00
2025-07-20 17:15:00+03:00
2025-07-20 03:00:00+03:00
2025-07-20 17:15:00+03:00
2025-08-07 17:15:00+03:00
2025-08-07 17:15:00+03:00
